In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Colors for each party
colors = {
    "People\'s Action Party": '#0000FF',  # Blue
    "Workers' Party": '#FF0000',  # Red
    "National Solidarity Party": '#00FF00',  # Green
    "Singapore Democratic Party": '#FFFF00',  # Yellow
    "Reform Party": '#FFA500',  # Orange
    "Singapore People's Party": '#800080',  # Purple
    "Progress Singapore Party": '#00FFFF',  # Cyan
    "People's Power Party": '#FF00FF',  # Magenta
    "Red Dot United": '#A52A2A',  # Brown
    "Singapore Democratic Alliance": '#808080',  # Gray
    "Singaporeans First": '#8B4513',  # SaddleBrown
    "Peoples Voice": '#FFD700',  # Gold
    "United People's Front": '#000000' # Black
}

def showparl(parties, colors, year, scenario, ax):
    # Remove parties with 0 seats
    parties = {k: v for k, v in parties.items() if v != 0}
    # Number of seats and angles for each party
    total_seats = sum(parties.values())
    angles = [party_seats / total_seats * np.pi for party_seats in parties.values()]

    # Cumulative angles for plotting
    cumulative_angles = np.cumsum(angles)

    # Start angle for each party
    start_angles = np.insert(cumulative_angles[:-1], 0, 0)

    # Plot each party as a segment in the semicircle
    for party, start_angle, angle in zip(parties.keys(), start_angles, angles):
        ax.bar(
            x=start_angle,
            height=1,
            width=angle,
            color=colors[party],
            edgecolor='black',
            align='edge',
            label=f"{party} ({parties[party]})"
        )

    # Set the plot limits and labels
    ax.set_ylim(0, 1)
    ax.set_yticklabels([])
    ax.set_xticks([])
    ax.set_theta_zero_location('W')
    ax.set_theta_direction(-1)
    ax.legend(loc='lower center')

    # Title
    ax.set_title(f'{year} Parliament Seats Distribution ({scenario})')

def return_actual_results(year):

    file_path = 'Actual_results/'+ str(year)+'-Table 1.csv'
    df = pd.read_csv(file_path)
    # Remove empty rows and columns
    df = df.dropna(how='all').dropna(axis=1, how='all')

    df.columns = df.iloc[0]
    df = df[1:]

    # Reset the index
    df.reset_index(drop=True, inplace=True)


    ## Data Cleaning 

    # Fill up rows, which dont have Constituency and Seats, with the respective constituency and seats
    for i in range(1, len(df)):
        if pd.isna(df.at[i, df.columns[0]]):

            df.loc[i, df.columns[0]] = df.loc[i-1, df.columns[0]]
            df.loc[i, df.columns[1]] = df.loc[i-1, df.columns[1]]

    df['%']=df['%'].str[:5]
    df['%']=df['%'].astype(float)
    df['Seats']=df['Seats'].astype(int)


    # Remove bug substring
    df['Party']=df['Party'].str.replace('\xa0','',regex=False)

    df.reset_index(drop=True, inplace=True)


    # Calculate Old Seats
    df['Old Seats']=0
    for i in range(0, len(df)):
        total_seats = df.iloc[i]['Seats']
        vote_percentage=df.iloc[i]['%']
        if vote_percentage> 50 or df.iloc[i]['Votes']=='Uncontested':
            df.at[i,'Old Seats'] = total_seats
        else:
            df.at[i,'Old Seats'] = 0

    # Group by Party and sum the Seats
    party_seats = df.groupby('Party')['Old Seats'].sum().reset_index()

    # Convert the result to a dictionary
    party_seats_dict = dict(party_seats.values)
    return party_seats_dict

def return_scenario_results(year,scenario):
    file_path = scenario +'_results/'+ str(year)+'_results.csv'
    df = pd.read_csv(file_path)

    # Group by Party and sum the Seats
    party_seat_change = df.groupby('Party')['Seat Change'].sum().reset_index()

    # Convert the result to a dictionary
    party_seat_change_dict = dict(party_seat_change.values)

    dict1 = party_seat_change_dict
    dict2 = return_actual_results(year)
    sum_dict = {k: dict1.get(k, 0) + dict2.get(k, 0) for k in set(dict1) | set(dict2)}
    return sum_dict


def output(year,scenario):
    fig, axs = plt.subplots(1, 2, figsize=(10, 5), subplot_kw={'projection': 'polar'})

    party_seats_dict_actual = return_actual_results(year)
    showparl(party_seats_dict_actual, colors, year, 'Actual', axs[0])

    party_seats_dict_scenario = return_scenario_results(year, scenario)
    showparl(party_seats_dict_scenario, colors, year, scenario, axs[1])

    plt.show()

import ipywidgets as widgets
from IPython.display import display

year_options = [1988,1991,1997,2001,2006,2011,2015,2020]
scenario_options = ['AntiPAP','ProPAP','ProWinner']
year_dropdown = widgets.Dropdown(options=year_options, value=2020, description='Year:')
scenario_dropdown = widgets.Dropdown(options=scenario_options, value='ProWinner', description='Scenario:')

ui = widgets.HBox([year_dropdown, scenario_dropdown])

out = widgets.interactive_output(output, {'year': year_dropdown, 'scenario': scenario_dropdown})
display(ui, out)


Output()